# two qubit vqe

In [28]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import warnings

warnings.filterwarnings('ignore')
np.random.seed(42)

In [29]:
I = np.eye(2, dtype=complex)
X = np.array([[0,1],[1,0]], dtype=complex)
Y = np.array([[0,-1j],[1j,0]], dtype=complex)
Z = np.array([[1,0],[0,-1]], dtype=complex)

def kron(A,B):
    """Kronecker product of two matrices"""
    return np.kron(A,B)

II = kron(I,I)
IX = kron(I,X)
IY = kron(I,Y)
IZ = kron(I,Z)
XI = kron(X,I)
YI = kron(Y,I)
ZI = kron(Z,I)
XX = kron(X,X)
YY = kron(Y,Y)
ZZ = kron(Z,Z)

print("pauli matrices:")
print(f"\nX=\n{X}")
print(f"\nZ=\n{Z}")
print(f"\nTwo-qubit Hilbert space dimension: {II.shape[0]}")
print(f"Basis states: |00>, |01>, |10>, |11>")

pauli matrices:

X=
[[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]]

Z=
[[ 1.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j]]

Two-qubit Hilbert space dimension: 4
Basis states: |00>, |01>, |10>, |11>


In [30]:
def Rx(theta):
    return np.array([
        [np.cos(theta/2), -1j*np.sin(theta/2)],
        [-1j*np.sin(theta/2), np.cos(theta/2)]
    ], dtype=complex)

def Ry(theta):
    return np.array([
        [np.cos(theta/2), -np.sin(theta/2)],
        [np.sin(theta/2), np.cos(theta/2)]
    ], dtype=complex)

def Rz(theta):
    return np.array([
        [np.exp(-1j*theta/2), 0],
        [0, np.exp(1j*theta/2)]
    ], dtype=complex)

def CNOT():
    return np.array([
        [1,0,0,0],
        [0,1,0,0],
        [0,0,0,1],
        [0,0,1,0]
    ], dtype=complex)

print("quantum gates defined:")
print("Rx(theta), Ry(theta), Rz(theta) - single qubit rotations")
print("CNOT - two-qubit entangling state")

quantum gates defined:
Rx(theta), Ry(theta), Rz(theta) - single qubit rotations
CNOT - two-qubit entangling state


In [31]:
def construct_hamiltonian(omega1, omega2, omega3, omega4, Vz, Vx):
    omega_II = (omega1 + omega2 + omega3 + omega4) / 4
    omega_ZI = (omega1 + omega2 - omega3 - omega4) / 4
    omega_IZ = (omega1 - omega2 + omega3 - omega4) / 4
    omega_ZZ = (omega1 - omega2 - omega3 + omega4) / 4

    H0 = omega_II*II + omega_ZI*ZI + omega_IZ*IZ + omega_ZZ*ZZ
    H1 = Vz*ZZ + Vx*XX
    H = H0+H1

    omega_coeffs = {
        'omega_II': omega_II,
        'omega_ZI': omega_ZI,
        'omega_IZ': omega_IZ,
        'omega_ZZ': omega_ZZ,
        'Vz': Vz,
        'Vx': Vx
    }
    
    return H, omega_coeffs

omega1 = 1.0
omega2 = 0.5
omega3 = 0.3
omega4 = 0.8
Vz = 0.2
Vx = 0.4

H, omega_coeffs = construct_hamiltonian(omega1, omega2, omega3, omega4, Vz, Vx)

print("hamiltonian:")
print(f"oemga1 = {omega1}")
print(f"oemga2 = {omega2}")
print(f"oemga3 = {omega3}")
print(f"oemga4 = {omega4}")
print(f"Vz = {Vz}")
print(f"Vx = {Vx}")

print(f"\nDerived Pauli Coefficients:")
print(f"  ω_II = {omega_coeffs['omega_II']:.4f}")
print(f"  ω_ZI = {omega_coeffs['omega_ZI']:.4f}")
print(f"  ω_IZ = {omega_coeffs['omega_IZ']:.4f}")
print(f"  ω_ZZ = {omega_coeffs['omega_ZZ']:.4f}")

print(f"\n hamiltonian matrix:")
print(H)
print(f"\n")
print(H.real)

hamiltonian:
oemga1 = 1.0
oemga2 = 0.5
oemga3 = 0.3
oemga4 = 0.8
Vz = 0.2
Vx = 0.4

Derived Pauli Coefficients:
  ω_II = 0.6500
  ω_ZI = 0.1000
  ω_IZ = 0.0000
  ω_ZZ = 0.2500

 hamiltonian matrix:
[[1.2+0.j 0. +0.j 0. +0.j 0.4+0.j]
 [0. +0.j 0.3+0.j 0.4+0.j 0. +0.j]
 [0. +0.j 0.4+0.j 0.1+0.j 0. +0.j]
 [0.4+0.j 0. +0.j 0. +0.j 1. +0.j]]


[[1.2 0.  0.  0.4]
 [0.  0.3 0.4 0. ]
 [0.  0.4 0.1 0. ]
 [0.4 0.  0.  1. ]]


In [32]:
evals_exact, evecs_exact = np.linalg.eigh(H)
E_exact = evals_exact[0]
psi_exact = evecs_exact[:, 0]

print("exact eigenvalues (sorted):")
for i, E in enumerate(evals_exact):
    print(f" E_{i} = {E:+.10f}")

print(f"\n exact ground state: {E_exact:.10f}")
print(f"\n exact ground state wavefunction:")
for i in range(4):
    basis_state = f"|{i:02b}>"
    amp = psi_exact[i]
    print(f"{basis_state}: {amp.real:+6f} {amp.imag:+6f}j")




exact eigenvalues (sorted):
 E_0 = -0.2123105626
 E_1 = +0.6123105626
 E_2 = +0.6876894374
 E_3 = +1.5123105626

 exact ground state: -0.2123105626

 exact ground state wavefunction:
|00>: -0.000000 +0.000000j
|01>: -0.615412 +0.000000j
|10>: +0.788205 +0.000000j
|11>: +0.000000 +0.000000j


In [ ]:
def prepare_ansatz(params):
    U1_q0 = Ry(params[0]) @ Rz(params[1])
    U1_q1 = Ry(params[2]) @ Rz(params[3])
    U1 = kron(U1_q0, U1_q1)
    U_cnot = CNOT()

    U2_q0 = Ry(params[4]) @ Rz(params[5])
    U2_q1 = Ry(params[6]) @ Rz(params[7])
    U2 = kron(U2_q0, U2_q1)

    U = U2 @ U_cnot @ 1

    return U

def prepare_initial_state():
    psi = np.zeros(4, dtype=complex)
    psi[0] = 1.0
    return psi

def apply_circuit(params):
    psi_0 = prepare_initial_state()
    U = prepare_ansatz(params)
    psi = U @ psi_0
    return psi

# test_params = np.random.uniform(-np.pi, np.pi, 8)
